In [1]:
from sklearn.preprocessing import PolynomialFeatures, FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import math
import pandas as pd
import numpy as np
import streamlit as st
from joblib import dump 
import joblib




df = pd.read_csv('dataset2.csv')

print(df.head(), df.shape)
# separation des features et de la variable cible
X = df.drop('charges', axis=1)
y = df[['charges']]
print(f'''verif des dimensions X et Y
      X (dataset sans la variable cible): {X.shape}
      Y (la variable cible) : {y.shape}''')


# division du dataset en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])
print(f''' verif du split 80 20 
80% du dataset : X train -> {X_train.shape}, Y train -> {y_train.shape}
# 20% du dataset : X test -> {X_test.shape}, Y test -> {y_test.shape}''')


   age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520 (1337, 7)
verif des dimensions X et Y
      X (dataset sans la variable cible): (1337, 6)
      Y (la variable cible) : (1337, 1)
 verif du split 80 20 
80% du dataset : X train -> (1136, 6), Y train -> (1136, 1)
# 20% du dataset : X test -> (201, 6), Y test -> (201, 1)


In [2]:
#Elasticnet
# préprocesseur pour les variables numériques
def log_transform(x):
    return np.log(x + 1)

log_transformer = FunctionTransformer(log_transform)

preprocessor_num = Pipeline(steps=[
    ('log', FunctionTransformer(log_transform)),
])

preprocessor_cat = Pipeline(steps=[
    ('encoder', OneHotEncoder())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', preprocessor_num, ['age', 'bmi', 'children']),
        ('cat', preprocessor_cat, ['region', 'sex', 'smoker'])
    ]
)

pipeline_lasso = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler()),
    ('regression', Lasso())
])

#test de différents hyperparam d'alpha pour le lasso
param_grid_lasso = {
    'regression__alpha': np.arange(7,10,0.1),
    'regression__precompute': [True],
    'regression__max_iter': [5000],
    
}

### grid search
grid_lasso = GridSearchCV(pipeline_lasso, param_grid_lasso, cv=5)


# eviter data leakage -> entraîner le pipeline sur les données d'entraînement 
grid_lasso.fit(X_train, y_train)
#puis predire y sur l'ensemble de test avec le meme pipeline
y_pred = grid_lasso.predict(X_test)
# print(y_pred)





/home/utilisateur/Bureau/ml/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 257432851.5586624, tolerance: 13153337.425098486
  model = cd_fast.enet_coordinate_descent_gram(
/home/utilisateur/Bureau/ml/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 500476679.9455986, tolerance: 12983618.917300116
  model = cd_fast.enet_coordinate_descent_gram(
/home/utilisateur/Bureau/ml/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 371280014.63212585, tolerance: 13974848.585028803
  model = cd_fast.enet_coordinate_descent_gram(
/home/utilisateur/Bureau/ml/lib/python3.10/site-

In [3]:
#test des les differents metriques sur modele elasticnet
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error du modèle elasticnet: {mse}')

r2 = r2_score(y_test, y_pred)

print(f'Coefficient de determination R² du modèle elasticnet: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle elasticnet: {rmse} \n rappel moyenne charge : 13279')

Mean Squared Error du modèle elasticnet: 13590867.153430084
Coefficient de determination R² du modèle elasticnet: 0.9062154146219557
Root Mean Squared Error (RMSE) du modèle elasticnet: 3686.579329599471 
 rappel moyenne charge : 13279


In [4]:
best_model = grid_lasso.best_estimator_
best_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('log',
                                                                   FunctionTransformer(func=<function log_transform at 0x7f42922af7f0>))]),
                                                  ['age', 'bmi', 'children']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder())]),
                                                  ['region', 'sex',
                                                   'smoker'])])),
                ('poly', PolynomialFeatures(include_bias=False)),
                ('scaler', StandardScaler()),
                ('regression',
                 Lasso(alpha=7.0, max_iter=5000, precompute=True))])

In [5]:
# extration  modèle Lasso
lasso_model = best_model.named_steps['regression']

# obtention des coefficients
coefficients = lasso_model.coef_
coefficients

for idx, coef in enumerate(sorted(coefficients, key=abs, reverse=True)):
    print(f"Coefficient {idx + 1}: {coef}")

Coefficient 1: -55006.406662746085
Coefficient 2: 19769.30887608532
Coefficient 3: 18800.964098093616
Coefficient 4: -18352.34107995047
Coefficient 5: -16204.44203451106
Coefficient 6: -15905.575176525303
Coefficient 7: 7773.186366791361
Coefficient 8: 5816.045129148156
Coefficient 9: -5255.49932942938
Coefficient 10: -4317.925125409152
Coefficient 11: -4088.551385724867
Coefficient 12: -3869.7769253455094
Coefficient 13: 1806.8584926890985
Coefficient 14: -1753.9170030249288
Coefficient 15: 1316.8198284488903
Coefficient 16: -1121.2222947776675
Coefficient 17: 653.8229149471638
Coefficient 18: -650.237538552091
Coefficient 19: 453.3412158233779
Coefficient 20: 338.62482233821254
Coefficient 21: 308.4563564478904
Coefficient 22: -305.20147523178696
Coefficient 23: 299.83754689749543
Coefficient 24: 207.0841153789487
Coefficient 25: 198.43029333909533
Coefficient 26: -175.19478057995798
Coefficient 27: 167.99990382589675
Coefficient 28: 156.20703159940993
Coefficient 29: -76.30922555345

In [6]:
new_data = pd.DataFrame({
    'age': [35, 40, 30, 60,60],  
    'sex': ['female', 'male', 'male', 'female','female'],  
    'bmi': [22.5, 30.0, 25.0, 47,50],
    'children': [1, 2, 0, 5,5],
    'smoker': ['yes', 'no', 'no', 'yes','yes'],  
    'region': ['southwest', 'northeast', 'southeast', 'southeast', 'southeast'],
})

# faire des prédictions avec le modèle optimisé
new_y_ped = best_model.predict(new_data)

# ajouter les prédictions à new_data
new_data['predicted_charges'] = new_y_ped

print(new_data)

   age     sex   bmi  children smoker     region  predicted_charges
0   35  female  22.5         1    yes  southwest       19070.268316
1   40    male  30.0         2     no  northeast        9673.987425
2   30    male  25.0         0     no  southeast        4727.669777
3   60  female  47.0         5    yes  southeast       58484.961989
4   60  female  50.0         5    yes  southeast       61143.119993


In [7]:
import pickle
with open('best_model.pkl', 'wb') as fichier:
    pickle.dump(best_model, fichier)